# ISOM 352 Applied Data Analytics with Coding
## Logistic Regression and Time Series Regression



In [11]:
# Install and import the library 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

plt.style.use('ggplot')

## In the context of Titanic, what's the main question?
### Q1: *Who `survived` Titanic tragedy?* Q2: "Women and Children first?"
- Step 1: Descriptive Analytics (numerically and graphically)
- Step 2: Simple logistic regression (as baseline)
- Step 3: Develop the best logistic regression model 
- Step 4: Intepret the impact of the variables 
- Step 5: Make prediction and evaluate the accuracy. 



### Step 0: Preprocessing the data
- proper data type
- missing values
- dummy variables

In [ ]:
# Step 0: Load and clean the dataset
df = pd.read_csv('titanic.csv')
df.info()

# Explore and process missing values
# 0.1 Cast object to category
for col in df.columns[df.dtypes == 'object']:
    df[col] = df[col].astype('category')

df.info()

# 0.2 Explore missing values
df = df.dropna(axis=0)
df.isna().sum()

# 03. Data manipulation: dummy variables for categorical variables are necessary for regression purpose
df_dummy = pd.get_dummies(data=df, drop_first=True, dtype='int8')
df_dummy.head()

### Step 1: Descriptive Analytics
### Step 2-3: Logistic Regression
### Step 4: Backward elimination

In [ ]:
# Step 3: Mutiple Regression model 
from scipy import stats

# Specify the predictor X and the target Y
y = df_dummy['survived']
x = df_dummy.drop(columns='survived')

# Add constant to the features
X = sm.add_constant(x)

# keep track of the variables in the model
variables = list(x.columns)

# Define the dummy groups
dummy_group = {'embark': [], 'class': []}
for var in variables:
    # let's initialize the dummy group
    for key in dummy_group.keys():
        if var.startswith(key):
            dummy_group[key].append(var)

print(f"variables: {variables}")
print(f"dummy_group: {dummy_group}")



In [ ]:
# Fit the full model
model = sm.Logit(y, X).fit()

# Get the highest pvalues from the model
pvalues = model.pvalues.drop('const')
max_pvalue = pvalues.max()
worst_var = pvalues.idxmax()

# if the worst variable is not significant, we will remove it
if max_pvalue > 0.05:
    prefix = worst_var.split("_")[0]
    
    # check if the worst variable is a dummy group    
    if prefix in dummy_group:
        # identify the dummy group that the worst_var belongs to
        vars_to_drop = dummy_group[prefix]

        # Compare the reduced model against the full model
        X_reduced = X.drop(columns=vars_to_drop)
        model_reduced = sm.Logit(y, X_reduced).fit()
        
        #TODO: Evaluate the significance of the reduced model
        lr_test = -2 * (model_reduced.llf - model.llf)
        f_pvalue = stats.chi2.sf(lr_test, len(vars_to_drop))
        
        # if the reduced model is not significant, we will remove the dummy group
        if f_pvalue >= 0.05:  
            X = X_reduced
            print(f"the dummy group {vars_to_drop} is excluded with f-pvalue {f_pvalue:.3f}")
        # otherwise, we keep the dummy group in the model but exclude from pvalue evaluation
        else: 
            print(f"the dummy group {vars_to_drop} is kept in the model")
            variables.remove(vars_to_drop)
    
    # if the worst variable is not a dummy group, we will remove it
    else: 
        X = X.drop(columns=worst_var)
        print(f"Variable {worst_var} is excluded with p-value {max_pvalue:.3f}")
    
else:
    print("All variables are significant")
    print(model.summary())

### Step 4: Interpret the impact of a feature
Instead of using the coefficients, we will be transform them into the odds ratio

In [ ]:
# Get the coefficients from the model
b_coef = model.params

# Exponentiate the parameters to get the odds ratio
import numpy as np
odds_ratios = np.exp(b_coef).round(3)
print(odds_ratios)

Q: How to interpre the odds ratio

## Part 1: Prediction and accuracy of prediction

### 1.1 Predict with Logistic model
- logit -> odds -> probability -> category

In [ ]:
# step 1: get the logit
logits = model.fittedvalues
# step 2: convert the logit to odds
odds = np.exp(logits)
# step 3: convert the odds to probability
y_prob = odds / (1 + odds)
# step 4: convert the probability to binary outcome
y_pred = (y_prob > 0.5).astype(int)

# display in a dataframe
pd.DataFrame(
    {
        'logit': logits,
        'odds': odds, 
        'probability': y_prob, 
        'prediction': y_pred
    }
)


### 3.2 Evaluation Metrics for Classification Models

- **Accuracy**: The proportion of correct predictions (both true positives and true negatives) among the total number of predictions. It measures overall correctness.
  
- **Precision**: The proportion of true positive predictions among all positive predictions. It measures how many of the predicted positives are actually correct.
  
- **Recall (Sensitivity)**: The proportion of true positive predictions among all actual positives. It measures how many of the actual positives were correctly identified.
  
- **F1-Score**: The harmonic mean of precision and recall, providing a balance between the two metrics.
  
- **Confusion Matrix**: A table showing the counts of true positives, false positives, true negatives, and false negatives, providing a comprehensive view of classification performance.


In [ ]:
# %pip install scikit-learn
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Evaluate the accuracy of prediction
accuracy = accuracy_score(y, y_pred)
precision = precision_score(y, y_pred)
recall = recall_score(y, y_pred)
f1 = f1_score(y, y_pred)

print(f"Accuracy: {accuracy:.2%}")
print(f"Precision: {precision:.2%}")
print(f"Recall: {recall:.2%}")
print(f"F1-Score: {f1:.2%}")

### 3.3 Visualize the results

In [ ]:
from sklearn.metrics import confusion_matrix

# plot the confusion matrix
def plot_confusion_matrix(
        y: np.ndarray, 
        y_pred: np.ndarray, 
        normalize: str | None = None) -> None:
    """
    Plot a confusion matrix with improved visualization.
    """
    cm = confusion_matrix(y, y_pred, normalize=normalize)
    sns.heatmap(cm, annot=True, cmap='Blues',
                xticklabels=['Negative', 'Positive'],
                yticklabels=['Negative', 'Positive'])
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()

# plot the probability distribution 
def plot_prediction_distribution(y: np.ndarray, y_prob: np.ndarray) -> None:
    """
    Plot the probability distribution of the predicted values.
    """
    plt.hist(y_prob[y == 0], bins=50, density=True, alpha=0.5,
             label='Died', color='red')
    plt.hist(y_prob[y == 1], bins=50, density=True, alpha=0.5,
             label='Survived', color='blue')
    plt.axvline(0.5, color='black', linestyle='--', alpha=0.5)
    plt.xlabel('Predicted probability')
    plt.ylabel('Density')
    plt.legend()
    plt.show()


In [ ]:
# plot the confusion matrix
plot_confusion_matrix(y, y_pred=y_pred, normalize='true')

# plot the probability distribution
plot_prediction_distribution(y, y_prob)

## Part 2: Handle Time Series Data 
- step 0: load the data and pre-processing
- Step 1: Plot the data and identify patterns
- Step 2: Choose appropriate model 
- Step 3: Make predictions and evaluate 

In [13]:
# import the libraries
# %pip install scikit-learn

from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_squared_error

In [ ]:
# Step 0: Load the data and pre-process data 
sales_data = pd.read_csv('RetailSales.csv')

# Explore the data
sales_data.info()
sales_data.head()



In [ ]:
# Rename the column to 'sales'
sales_data = sales_data.rename(columns={'MRTSSM4453USN': 'sales'})

# Convert to datetime
sales_data['DATE'] = pd.to_datetime(sales_data['DATE'])

# Set datetime as index
sales_data = sales_data.set_index('DATE')
sales_data.index.freq = 'MS'
sales_data.index

### 2.1: Decompose the data

In [ ]:
# Plot the data
sales_data.plot()

In [ ]:
# decompose the time series data

# plot the decomposed data


### 2.2: Choose an appropriate model
SARIMA (Seasonal AutoRegressive Integrated Moving Average) Model:
 - AR(p): AutoRegressive component - uses past values to predict future values
 - I(d): Integrated component - differencing to make the time series stationary
 - MA(q): Moving Average component - uses past forecast errors in the model
 - Seasonal component (P,D,Q,s): Captures seasonal patterns in the data
 - Parameters (p,d,q)(P,D,Q,s) must be specified when fitting the model
 - p: order of the autoregressive part
 - d: degree of differencing required for stationarity
 - q: order of the moving average part
 - P: seasonal autoregressive order
 - D: seasonal differencing order
 - Q: seasonal moving average order
 - s: length of the seasonal cycle

In [ ]:
# For simplicity and as an example, we will choose a basic set of parameters.

# Create the model and fit data
sales_sarima = SARIMAX(
    sales_data, 
    order=(2, 1, 1), 
    seasonal_order=(1, 1, 1, 12)
    ).fit()


# Show the model summary 
print(sales_sarima.summary())


### 3.3: Make Predictions
Prediction is a generally term that can apply to within or out of data range

In [ ]:
# Get prediction and plot the data
fitted_values = sales_sarima.get_prediction()
sales_pred = fitted_values.predicted_mean

# Evaluate the accuracy
mse = mean_squared_error(sales_data, sales_pred)
rmse = np.sqrt(mse)

print(f"RMSE: {rmse:.2f}")

# Visualize the prediction
plt.figure(figsize=(12,6))

plt.figure(figsize=(12,6))
plt.plot(sales_data, label='Actual Sales')
plt.plot(sales_pred, label='Predicted Sales')

# # Get the confidence interval
# conf_lower = fitted_values.conf_int()['lower sales']
# conf_upper = fitted_values.conf_int()['upper sales']
# plt.fill_between(x=sales_pred.index,
#                  y1=conf_lower,
#                  y2=conf_upper,
#                  color='gray')
# plt.legend()
# plt.show()



### Step 4: Make Forecasts
Forecast tyically refers to beyond current time horizon 

In [ ]:
plt.plot(sales_data, label='Actual Sales')

# Make forecast and plot the confidence interval
forecast_results = sales_sarima.get_forecast(steps=36)
forecast = forecast_results.predicted_mean
plt.plot(forecast, label='forecast')

# Confidence interval
conf_lower = forecast_results.conf_int()['lower sales']
conf_upper = forecast_results.conf_int()['upper sales']
plt.fill_between(x=forecast.index,
                 y1=conf_lower,
                 y2=conf_upper,
                 color='gray')
plt.legend()
plt.xlim(left=pd.to_datetime('2018-01-01'))
plt.show()